In [1]:
import faiss # make faiss available
import numpy as np
import json
import random
from PIL import Image
import matplotlib.pyplot as plt
import time
import pandas as pd 
import json
%matplotlib inline

In [2]:
image_paths=[]
with open("../image-files/geological_map.json", 'r', encoding='utf-8') as f:
    image_paths=json.load(f)
print(len(image_paths))

embedding = np.load("../encoders/geological_embed.npy")

29998


In [3]:
random_img_index=random.randint(0, len(image_paths)-1)
xq=embedding[random_img_index]
xb=embedding
print(xb.shape)
print(xq.shape)
xq=np.reshape(xq, (1, 576))
print(xq.shape)

(29998, 576)
(576,)
(1, 576)


In [ ]:
input_image_path=image_paths[random_img_index]
input_img = Image.open("../"+input_image_path).convert("RGB")
plt.imshow(input_img)
plt.show()

In [ ]:
index = faiss.IndexFlatL2(576)# build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)
k=10
Dist, Indices = index.search(xq, k)     # actual search
print(Indices)                   # neighbors of the 5 first queries


In [ ]:
indices = Indices[0]
print("total indices: ", len(indices))
for index in indices:
    # img_name = str(index - 1) + ".jpg"
    # print(img_name)
    img_path = image_paths[index]
    print(img_path)
    img = Image.open("../"+img_path).convert("RGB")
    plt.imshow(img)
    plt.show()

In [ ]:
#IVF-L2
import time
nlist = 100
k = 100
quantizer = faiss.IndexFlatL2(576)  # the other index
index = faiss.IndexIVFFlat(quantizer, 576, nlist, faiss.METRIC_L2)
# here we specify METRIC_L2, by default it performs inner-product search

assert not index.is_trained
index.train(xb)
assert index.is_trained

index.add(xb)                  # add may be a bit slower as well

start=time.time()
D, I = index.search(xq, k)     # actual search
# print(I[-5:])# neighbors of the 5 last queries
end=time.time()
print(end-start)
print(I)
index.nprobe = 10 # default nprobe is 1, try a few more
start=time.time()
D, I = index.search(xq, k)
end=time.time()
print(end-start)
print(I)                  # neighbors of the 5 last queries

In [ ]:
indices = I[0]
print("total indices: ", len(indices))
for index in indices:
    # img_name = str(index - 1) + ".jpg"
    # print(img_name)
    img_path = image_paths[index]
    print(img_path)
    img = Image.open("../"+img_path).convert("RGB")
    plt.imshow(img)
    plt.show()

In [ ]:
import pandas as pd


In [4]:
df=pd.read_csv("../knn-results/cosine-results.csv")
df.head()

,testing_image,time_taken,indices_list,output_indices
0,../geological_similarity/schist/TEJ1Y.jpg,0.106924,NaN,"[25578, 25754, 29111, 29065, 26147, 29888, 252..."
1,../geological_similarity/marble/13P1E.jpg,0.068276,NaN,"[13641, 13191, 10152, 10489, 11181, 11653, 119..."
2,../geological_similarity/andesite/RRGXD.jpg,0.067910,NaN,"[17741, 2403, 4762, 149, 4965, 1685, 842, 1399..."
3,../geological_similarity/schist/PEVYX.jpg,0.068621,NaN,"[9801, 9209, 6398, 6674, 9056, 7528, 7636, 813..."
4,../geological_similarity/schist/66ZT7.jpg,0.067650,NaN,"[25855, 27277, 5806, 8640, 6559, 25850, 8451, ..."


In [8]:
for i, obj in enumerate(image_paths):
    image_paths[i]=obj.replace("\\", "/")
    image_paths[i]="../"+image_paths[i]


In [18]:
def create_index(xb, nlist=100, nprobe=1):
    nlist = 100
    quantizer = faiss.IndexFlatIP(576)  # the other index
    index = faiss.IndexIVFFlat(quantizer, 576, nlist, faiss.METRIC_L2)
    # here we specify METRIC_L2, by default it performs inner-product search

    assert not index.is_trained
    index.train(xb)
    assert index.is_trained

    index.add(xb)
    return index

In [19]:
def search_index(index, xq, k):
    start=time.time()
    D, I = index.search(xq, k)     # actual search
    end=time.time()
    total_time=end-start
    return total_time, D, I[0]

In [20]:
def IVF_search(k, results_df, nlist=100, nprobe=1,):
    xb=embedding
    start=time.time()
    index=create_index(xb, nlist, nprobe)
    end=time.time()
    time_index=end-start
    for i, obj in enumerate(df["testing_image"]):
        xq=embedding[image_paths.index(obj)]
        xq=np.reshape(xq, (1, 576))
        total_time, D, indices=search_index(index, xq, k)
        if len(indices)!=0:
            acc=len(list(set(indices).intersection(df["output_indices"][i])))/len(indices)
        else:
            acc=0
        results_df=results_df.append({"testing_image":obj, "nlist":nlist, "nprobe":nprobe, "search_time":total_time, "output_indices":indices, "acc":acc, "index_time":time_index}, ignore_index=True)
    return results_df

In [21]:
results={"testing_image":[], "nlist":[], "nprobe":[], "output_indices":[], "acc":[]}
results_df=pd.DataFrame(results)
results_df.head()

,testing_image,nlist,nprobe,output_indices,acc


In [22]:
nlist=[50, 100, 200, 400, 800, 1600]
nprobe=[1, 4, 8, 32, 64, 128, 256, 512, 1024]
for nl in nlist:
    for npr in nprobe:
        if npr>=nl:
            continue
        results_df=IVF_search(50,results_df, nl, npr)
        print(nl, npr, "done")
results_df.info()

50 1 done
50 4 done
50 8 done
50 32 done
100 1 done
100 4 done
100 8 done
100 32 done
100 64 done
200 1 done
200 4 done
200 8 done
200 32 done
200 64 done
200 128 done
400 1 done
400 4 done
400 8 done
400 32 done
400 64 done
400 128 done
400 256 done
800 1 done
800 4 done
800 8 done
800 32 done
800 64 done
800 128 done
800 256 done
800 512 done
1600 1 done
1600 4 done
1600 8 done
1600 32 done
1600 64 done
1600 128 done
1600 256 done
1600 512 done
1600 1024 done
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   testing_image   3900 non-null   object 
 1   nlist           3900 non-null   float64
 2   nprobe          3900 non-null   float64
 3   output_indices  3900 non-null   object 
 4   acc             3900 non-null   float64
 5   index_time      3900 non-null   float64
 6   search_time     3900 non-null   float64
dtypes: float64(5), obj

In [23]:
results_df.describe()

,nlist,nprobe,acc,index_time,search_time
count,3900.000000,3900.000000,3900.000000,3900.000000,3900.000000
mean,653.846154,100.461538,0.745200,0.823175,0.021106
std,575.353479,194.071513,0.301775,0.082671,0.005135
min,50.000000,1.000000,0.020000,0.694064,0.013004
25%,200.000000,4.000000,0.535000,0.778003,0.017982
50%,400.000000,32.000000,0.940000,0.804008,0.020996
75%,800.000000,128.000000,1.000000,0.838007,0.024003
max,1600.000000,1024.000000,1.000000,1.145166,0.177998


In [24]:
results_df[results_df["testing_image"]==results_df["testing_image"][1021]]

,testing_image,nlist,nprobe,output_indices,acc,index_time,search_time
21,../geological_similarity/gneiss/SKDTL.jpg,50.0,1.0,"[8944, 5935, 8062, 6018, 7259, 8096, 6356, 280...",0.98,0.833307,0.023988
121,../geological_similarity/gneiss/SKDTL.jpg,50.0,4.0,"[8944, 5935, 8062, 6018, 7259, 8096, 6356, 280...",0.98,0.882787,0.029005
221,../geological_similarity/gneiss/SKDTL.jpg,50.0,8.0,"[8944, 5935, 8062, 6018, 7259, 8096, 6356, 280...",0.98,0.838002,0.024000
321,../geological_similarity/gneiss/SKDTL.jpg,50.0,32.0,"[8944, 5935, 8062, 6018, 7259, 8096, 6356, 280...",0.98,0.876992,0.025002
421,../geological_similarity/gneiss/SKDTL.jpg,100.0,1.0,"[8944, 5935, 8062, 6018, 7259, 8096, 6356, 280...",0.98,0.694064,0.022996
521,../geological_similarity/gneiss/SKDTL.jpg,100.0,4.0,"[8944, 5935, 8062, 6018, 7259, 8096, 6356, 280...",0.98,0.773045,0.021995
621,../geological_similarity/gneiss/SKDTL.jpg,100.0,8.0,"[8944, 5935, 8062, 6018, 7259, 8096, 6356, 280...",0.98,0.977002,0.025999
721,../geological_similarity/gneiss/SKDTL.jpg,100.0,32.0,"[8944, 5935, 8062, 6018, 7259, 8096, 6356, 280...",0.98,1.145166,0.027009
821,../geological_similarity/gneiss/SKDTL.jpg,100.0,64.0,"[8944, 5935, 8062, 6018, 7259, 8096, 6356, 280...",0.98,0.781997,0.025002
921,../geological_similarity/gneiss/SKDTL.jpg,200.0,1.0,"[8944, 5935, 8062, 6018, 7259, 8096, 6356, 280...",0.98,0.824998,0.023000


In [25]:
results_df.to_csv("../faiss-results/l2-faiss.csv", index=False)

In [ ]:
nbits = 576*4  # resolution of bucketed vectors
# initialize index and add vectors
index = faiss.IndexLSH(576, nbits)
index.add(xb)
# and search
D, I = index.search(xq, 10)

In [ ]:
def create_LSHBuckets(xb, nbit_factor):
    nbits = 576*nbit_factor  # resolution of bucketed vectors
    # initialize index and add vectors
    index = faiss.IndexLSH(576, nbits)
    index.add(xb)
    return index

In [ ]:
def LSH_search(k, results_df, nbit_factor):
    xb=embedding
    start=time.time()
    index=create_LSHBuckets(xb,nbit_factor)
    end=time.time()
    time_index=end-start
    for i, obj in enumerate(df["testing_image"]):
        xq=embedding[image_paths.index(obj)]
        xq=np.reshape(xq, (1, 576))
        total_time, D, indices=search_index(index, xq, k)
        if len(indices)!=0:
            acc=len(list(set(indices).intersection(df["output_indices"][i])))/len(indices)
        else:
            acc=0
        results_df=results_df.append({"testing_image":obj, "nbit-factor":nbit_factor, "search_time":total_time, "output_indices":indices, "acc":acc, "index_time":time_index}, ignore_index=True)
    return results_df

In [6]:
results={"testing_image":[], "output_indices":[]}
results_df=pd.DataFrame(results)
results_df.head()

,testing_image,output_indices


In [ ]:
nbit=[1, 2, 4, 8, 16, 32]
for nl in nbit:
    results_df=LSH_search(50,results_df, nl)
    print(nl, "done")
results_df.info()

In [ ]:
results_df.describe()

In [ ]:
results_df[0:100]

In [ ]:
results_df.to_csv("../LSH-results/LSH-faiss.csv", index=False)

In [11]:
def L2_index(results_df):
    xb=embedding
    index = faiss.IndexFlatL2(576)# build the index
    print(index.is_trained)
    index.add(xb) 
    for i, obj in enumerate(df["testing_image"]):
        xq=embedding[image_paths.index(obj)]
        xq=np.reshape(xq, (1, 576))
        total_time, D, indices=search_index(index, xq, 50)
        results_df=results_df.append({"testing_image":obj,  "search_time":total_time, "output_indices":indices}, ignore_index=True)
    return results_df
df=L2_index(results_df)

True


In [12]:
df.to_csv("../faiss-results/Flat_L2.csv")